In [ ]:
from scipy.optimize import curve_fit
import hyperspy.api as hys
import numpy as np
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
from tqdm import tqdm
plt.rcParams['font.family'] = 'Cambria'

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
spectrum_adr = []

In [ ]:
spectrum_adr.extend(tkf.askopenfilenames())
print(*spectrum_adr, sep="\n")

In [ ]:
num_img = len(spectrum_adr)
print(num_img)

In [ ]:
si_data = []
e_ranges = []
steps = []
for i in range(num_img):
    spectrum = hys.load(spectrum_adr[i], signal_type="EELS")
    step = spectrum.axes_manager[2].scale
    offset = spectrum.axes_manager[2].offset
    e_size = spectrum.axes_manager[2].size
    print(step, offset, e_size)
    e_range = np.arange(offset, offset+e_size*step, step)
    si_data.append(spectrum.data.clip(min=0.0))
    e_ranges.append(e_range)
    steps.append(step)

In [ ]:
def gauss1(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss2(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss3(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss4(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss5(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def two_gauss(x, a1, a2, c1, c2, sigma1, sigma2):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)

def three_gauss(x, a1, a2, a3, c1, c2, c3, sigma1, sigma2, sigma3):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)+gauss3(x, a3, c3, sigma3)

def four_gauss(x, a1, a2, a3, a4, c1, c2, c3, c4, sigma1, sigma2, sigma3, sigma4):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)+gauss3(x, a3, c3, sigma3)+gauss4(x, a4, c4, sigma4)

def five_gauss(x, a1, a2, a3, a4, a5, c1, c2, c3, c4, c5, sigma1, sigma2, sigma3, sigma4, sigma5):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)+gauss3(x, a3, c3, sigma3)+gauss4(x, a4, c4, sigma4)+gauss5(x, a5, c5, sigma5)

In [ ]:
fit1_bound = (0.0, 1.0, 14.0, 18.0, 0.0, 3.0)
fit2_bound = (0.0, 1.0, 21.0, 25.0, 0.0, 3.0)
fit3_bound = (0.0, 1.0, 27.0, 29.0, 0.0, 3.0)

bound_left = [fit1_bound[0], fit2_bound[0], fit3_bound[0], 
              fit1_bound[2], fit2_bound[2], fit3_bound[2],
              fit1_bound[4], fit2_bound[4], fit3_bound[4]]

bound_right = [fit1_bound[1], fit2_bound[1], fit3_bound[1], 
              fit1_bound[3], fit2_bound[3], fit3_bound[3],
              fit1_bound[5], fit2_bound[5], fit3_bound[5]]

In [ ]:
start_ev = 10.0
end_ev = 35.0

pad = 50

pbar = tqdm(total=num_img)

total_maps = []
area_maps = []
for k, SI in enumerate(si_data):
    e_range = e_ranges[k]
    start_ind = find_nearest(e_range, start_ev)
    end_ind = find_nearest(e_range, end_ev)
    print(start_ev, start_ind)
    print(end_ev, end_ind)
    
    params_map = []
    fit_area = []
    for i in range(SI.shape[0]):
        for j in range(SI.shape[1]):
            signal = SI[i, j]
            e_range_int = e_range[start_ind:end_ind]
            signal_int = signal[start_ind:end_ind]
            signal_int = signal_int / np.max(signal_int)

            slope = (signal_int[-1] - signal_int[0]) / (e_range_int[-1] - e_range_int[0])
            intercept = signal_int[0] - slope*e_range_int[0]
            bg_line = slope*e_range_int+intercept

            signal_int_bg_removed = signal_int - bg_line
            signal_int_bg_removed = np.append(np.zeros(pad), signal_int_bg_removed)
            signal_int_bg_removed = np.append(signal_int_bg_removed, np.zeros(pad))
            signal_int_bg_removed = signal_int_bg_removed / np.max(signal_int_bg_removed)
            e_range_int_bg_removed = np.arange(start_ev-pad*steps[k], end_ev+pad*steps[k], steps[k])

            bg_line_bg_removed = slope*e_range_int_bg_removed+intercept / np.max(signal_int_bg_removed)

            popt, pcov = curve_fit(three_gauss, e_range_int_bg_removed, signal_int_bg_removed, 
                                   bounds=(bound_left, bound_right))
            
            fit1 = gauss1(e_range_int_bg_removed, popt[0], popt[3], popt[6])
            fit2 = gauss2(e_range_int_bg_removed, popt[1], popt[4], popt[7])
            fit3 = gauss3(e_range_int_bg_removed, popt[2], popt[5], popt[8])

            fit1_area = np.trapz(fit1, e_range_int_bg_removed)
            fit2_area = np.trapz(fit2, e_range_int_bg_removed)
            fit3_area = np.trapz(fit3, e_range_int_bg_removed)
    
            fit_area.append([fit1_area, fit2_area, fit3_area])
            params_map.append(popt)
    params_map = np.asarray(params_map).reshape(SI.shape[0], SI.shape[1], -1)
    fit_area = np.asarray(fit_area).reshape(SI.shape[0], SI.shape[1], -1)
    total_maps.append(params_map)
    area_maps.append(fit_area)
    pbar.update(1)
pbar.close()

In [ ]:
fit1_bound = (0.0, 1.0, 14.0, 16.0, 0.0, 3.0)
fit2_bound = (0.0, 1.0, 19.0, 23.0, 0.0, 3.0)
fit3_bound = (0.0, 1.0, 25.0, 30.0, 0.0, 3.0)
fit4_bound = (0.0, 1.0, 38.0, 42.0, 0.0, 3.0)

bound_left = [fit1_bound[0], fit2_bound[0], fit3_bound[0], fit4_bound[0], 
              fit1_bound[2], fit2_bound[2], fit3_bound[2], fit4_bound[2],
              fit1_bound[4], fit2_bound[4], fit3_bound[4], fit4_bound[4]]

bound_right = [fit1_bound[1], fit2_bound[1], fit3_bound[1], fit4_bound[1], 
              fit1_bound[3], fit2_bound[3], fit3_bound[3], fit4_bound[3],
              fit1_bound[5], fit2_bound[5], fit3_bound[5], fit4_bound[5]]

In [ ]:
start_ev = 10.0
end_ev = 43.0

pad = 150

pbar = tqdm(total=num_img)

total_maps = []
area_maps = []
for k, SI in enumerate(si_data):
    e_range = e_ranges[k]
    start_ind = find_nearest(e_range, start_ev)
    end_ind = find_nearest(e_range, end_ev)
    print(start_ev, start_ind)
    print(end_ev, end_ind)
    
    params_map = []
    fit_area = []
    for i in range(SI.shape[0]):
        for j in range(SI.shape[1]):
            signal = SI[i, j]
            e_range_int = e_range[start_ind:end_ind]
            signal_int = signal[start_ind:end_ind]
            signal_int = signal_int / np.max(signal_int)

            slope = (signal_int[-1] - signal_int[0]) / (e_range_int[-1] - e_range_int[0])
            intercept = signal_int[0] - slope*e_range_int[0]
            bg_line = slope*e_range_int+intercept

            signal_int_bg_removed = signal_int - bg_line
            signal_int_bg_removed = np.append(np.zeros(pad), signal_int_bg_removed)
            signal_int_bg_removed = np.append(signal_int_bg_removed, np.zeros(pad))
            signal_int_bg_removed = signal_int_bg_removed / np.max(signal_int_bg_removed)
            e_range_int_bg_removed = np.arange(start_ev-pad*steps[k], end_ev+pad*steps[k], steps[k])

            bg_line_bg_removed = slope*e_range_int_bg_removed+intercept / np.max(signal_int_bg_removed)
            
            try:
                popt, pcov = curve_fit(four_gauss, e_range_int_bg_removed, signal_int_bg_removed, 
                                   bounds=(bound_left, bound_right))
            except RuntimeError:
                print(spectrum_adr[k])
                print((i+1), (j+1), "pixel error")
            
            fit1 = gauss1(e_range_int_bg_removed, popt[0], popt[4], popt[8])
            fit2 = gauss2(e_range_int_bg_removed, popt[1], popt[5], popt[9])
            fit3 = gauss3(e_range_int_bg_removed, popt[2], popt[6], popt[10])
            fit4 = gauss4(e_range_int_bg_removed, popt[3], popt[7], popt[11])
            fit1_area = np.trapz(fit1, e_range_int_bg_removed)
            fit2_area = np.trapz(fit2, e_range_int_bg_removed)
            fit3_area = np.trapz(fit3, e_range_int_bg_removed)
            fit4_area = np.trapz(fit4, e_range_int_bg_removed)
    
            fit_area.append([fit1_area, fit2_area, fit3_area, fit4_area])
            params_map.append(popt)
    params_map = np.asarray(params_map).reshape(SI.shape[0], SI.shape[1], -1)
    fit_area = np.asarray(fit_area).reshape(SI.shape[0], SI.shape[1], -1)
    total_maps.append(params_map)
    area_maps.append(fit_area)
    pbar.update(1)
pbar.close()

In [ ]:
for i in range(9):
    fig, ax = plt.subplots(1, num_img, figsize=(10, 7))
    for j in range(num_img):
        ax[j].imshow(total_maps[j][:, :, i], cmap="inferno")
        ax[j].axis("off")
    fig.tight_layout()
    plt.show()

In [ ]:
for i in range(4):
    fig, ax = plt.subplots(1, num_img, figsize=(10, 7))
    for j in range(num_img):
        ax[j].imshow(area_maps[j][:, :, i], cmap="inferno")
        ax[j].axis("off")
    fig.tight_layout()
    plt.show()